### 목차

전체 코드

1. 필요한 라이브러리 설치
2. 라이브러리 임포트 및 장치 설정
3. 데이터 준비
4. 데이터 전처리
5. 데이터셋 및 데이터로더 생성
6. 모델 로드 및 설정
7. 모델 학습
8. 모델 저장
9. 챗봇 실행을 위한 모델 로드
10. 챗봇 응답 생성 함수 정의
11. 챗봇 실행

추가 고려 사항

결론

#### 1. 필요한 라이브러리 설치
- 먼저, 필요한 라이브러리를 설치합니다. 이 셀을 실행하여 transformers, torch, sentencepiece 등을 설치하세요.
- !pip install transformers torch sentencepiece

#### 2. 라이브러리 임포트 및 장치 설정
- 필요한 라이브러리를 임포트하고, GPU 사용 가능 여부를 확인합니다.

In [4]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import TensorDataset, DataLoader

# 장치 설정 (GPU 사용 가능 시 GPU, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")


사용 중인 장치: cuda


#### 3. 데이터 준비
- 음식 추천 챗봇을 위한 질문-답변 데이터를 준비
- 실제 프로젝트에서는 더 많은 데이터 필요.

In [2]:

train_data = [
    {"input": "배가 고픈데 뭐 먹을까?", "output": "한식은 어때요? 따뜻한 국밥이 좋을 것 같아요."},
    {"input": "점심 추천해줘", "output": "가벼운 샐러드나 샌드위치는 어떠세요?"},
    {"input": "저녁 뭐 먹을지 고민돼", "output": "맛있는 파스타나 스테이크는 어떠신가요?"},
    {"input": "달콤한 디저트 먹고 싶어", "output": "초콜릿 케이크나 아이스크림을 추천드려요."},
    {"input": "시원한 거 마시고 싶어", "output": "아이스 아메리카노나 레몬 에이드를 드셔보세요."},
    {"input": "매운 음식이 땡겨", "output": "매운 떡볶이나 불닭볶음면은 어떠세요?"},
    {"input": "간단하게 먹을 거 없어?", "output": "김밥이나 샌드위치를 드셔보세요."},
    {"input": "한국 전통 음식 추천해줘", "output": "비빔밥이나 불고기를 추천합니다."},
    {"input": "중국 음식이 먹고 싶어", "output": "짜장면이나 탕수육은 어떠세요?"},
    {"input": "일식 좋아해", "output": "스시나 우동을 드셔보세요."},
    {"input": "양식 먹고 싶어", "output": "스테이크나 피자를 추천합니다."},
    {"input": "채식주의자야", "output": "채소 샐러드나 두부 요리를 드셔보세요."},
    {"input": "디저트 카페 추천해줘", "output": "근처에 있는 베이커리 카페는 어떠세요?"},
    {"input": "건강식이 필요해", "output": "현미밥과 야채로 구성된 식단을 추천합니다."},
    {"input": "해산물 요리 먹고 싶어", "output": "회나 해물파전은 어떠세요?"},
    {"input": "친구들이랑 먹을 거 추천해줘", "output": "치킨이나 피자를 시켜 드세요."},
    {"input": "새로운 맛집 없어?", "output": "요즘 핫한 수제 버거 가게를 추천합니다."},
    {"input": "간식 추천해줘", "output": "과일이나 요거트는 어떠신가요?"},
    {"input": "더운 날씨에 뭐 먹지?", "output": "시원한 냉면이나 팥빙수를 드셔보세요."},
    {"input": "감기 걸렸어", "output": "따뜻한 죽이나 수프를 드시는 게 좋겠어요."}
]


#### 4. 데이터 전처리
- 입력과 출력 텍스트를 하나의 시퀀스로 결합하고, 레이블을 생성
- 입력 부분은 손실 계산에서 제외하기 위해 -100으로 마스킹 
- 또한, max_length를 명시적으로 설정하여 경고를 방지
##### 설명:
- 패딩 토큰과 종료 토큰 구분: tokenizer.pad_token과 tokenizer.eos_token이 동일한지 확인하고, 동일하다면 pad_token을 <pad>로 추가하여 구분, 
- 이는 attention_mask가 정확하게 생성되도록 함
- 토크나이징 시 max_length 설정: max_length=128을 설정하여 입력 시퀀스의 최대 길이를 제한
- 이는 경고를 방지하고, 모델이 예상치 못한 긴 입력을 처리하지 않도록 함
- 레이블 마스킹: 입력 부분을 -100으로 마스킹하여 모델이 손실을 계산할 때 해당 부분을 무시하도록 처리

In [5]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>'
)

# pad_token이 없는 경우 추가
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})

inputs = []
labels = []

for pair in train_data:
    input_text = f"질문: {pair['input']}\n답변: "
    output_text = pair['output'] + tokenizer.eos_token
    full_text = input_text + output_text
    inputs.append(full_text)

# 토크나이즈 및 텐서 변환 (max_length를 명시적으로 설정)
encodings = tokenizer(
    inputs,
    padding=True,
    truncation=True,
    max_length=128,  # 필요에 따라 조정 가능
    return_tensors='pt'
)

input_ids = encodings.input_ids
attention_mask = encodings.attention_mask
labels = input_ids.clone()

# 입력 부분 마스킹
for i in range(len(train_data)):
    input_text = f"질문: {train_data[i]['input']}\n답변: "
    input_ids_i = tokenizer.encode(input_text, add_special_tokens=False)
    input_length = len(input_ids_i)
    labels[i, :input_length] = -100  # 입력 부분 마스킹


c:\ProgramData\anaconda3\envs\gnn_snn_torch\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### 5. 데이터셋 및 데이터로더 생성
- 파이토치의 TensorDataset과 DataLoader를 사용하여 데이터를 관리

In [6]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(input_ids, encodings.attention_mask, labels)
loader = DataLoader(dataset, batch_size=4, shuffle=True)


#### 6. 모델 로드 및 설정
- 사전 학습된 KoGPT2 모델을 로드하고, 패딩 토큰을 반영하도록 설정

##### 설명:
- 토큰 임베딩 크기 조정: 패딩 토큰을 추가한 후, 모델의 토큰 임베딩 크기를 조정하여 새로운 토큰을 반영
- 이를 통해 패딩 토큰을 제대로 인식할 수 있도록 처리

In [7]:
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.resize_token_embeddings(len(tokenizer))  # pad_token 추가 후 토큰 임베딩 크기 조정
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)


c:\ProgramData\anaconda3\envs\gnn_snn_torch\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### 7. 모델 학습
- 모델을 학습
- 에포크 수를 늘려 모델이 데이터를 충분한 학습 필요
##### 설명:
- 에포크 증가: 에포크 수를 늘려 모델이 데이터를 충분히 학습하도록 처리 필요
- 손실 값 모니터링: 각 에포크마다 평균 손실 값을 출력하여 학습 상태를 모니터링

In [8]:
epochs = 100  # 에포크 수 조정 가능

for epoch in range(epochs):
    print(f"에포크 {epoch+1}/{epochs} 진행 중...")
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        input_ids_batch = batch[0].to(device)
        attention_mask_batch = batch[1].to(device)
        labels_batch = batch[2].to(device)

        outputs = model(
            input_ids=input_ids_batch,
            attention_mask=attention_mask_batch,
            labels=labels_batch
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"에포크 {epoch+1} 완료, 평균 손실 값: {avg_loss:.4f}")


에포크 1/100 진행 중...


c:\ProgramData\anaconda3\envs\gnn_snn_torch\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


에포크 1 완료, 평균 손실 값: 4.9320
에포크 2/100 진행 중...
에포크 2 완료, 평균 손실 값: 1.2204
에포크 3/100 진행 중...
에포크 3 완료, 평균 손실 값: 0.3581
에포크 4/100 진행 중...
에포크 4 완료, 평균 손실 값: 0.2390
에포크 5/100 진행 중...
에포크 5 완료, 평균 손실 값: 0.1685
에포크 6/100 진행 중...
에포크 6 완료, 평균 손실 값: 0.0716
에포크 7/100 진행 중...
에포크 7 완료, 평균 손실 값: 0.0680
에포크 8/100 진행 중...
에포크 8 완료, 평균 손실 값: 0.0482
에포크 9/100 진행 중...
에포크 9 완료, 평균 손실 값: 0.0529
에포크 10/100 진행 중...
에포크 10 완료, 평균 손실 값: 0.0653
에포크 11/100 진행 중...
에포크 11 완료, 평균 손실 값: 0.0292
에포크 12/100 진행 중...
에포크 12 완료, 평균 손실 값: 0.0296
에포크 13/100 진행 중...
에포크 13 완료, 평균 손실 값: 0.0240
에포크 14/100 진행 중...
에포크 14 완료, 평균 손실 값: 0.0163
에포크 15/100 진행 중...
에포크 15 완료, 평균 손실 값: 0.0212
에포크 16/100 진행 중...
에포크 16 완료, 평균 손실 값: 0.0359
에포크 17/100 진행 중...
에포크 17 완료, 평균 손실 값: 0.0361
에포크 18/100 진행 중...
에포크 18 완료, 평균 손실 값: 0.0214
에포크 19/100 진행 중...
에포크 19 완료, 평균 손실 값: 0.0219
에포크 20/100 진행 중...
에포크 20 완료, 평균 손실 값: 0.0053
에포크 21/100 진행 중...
에포크 21 완료, 평균 손실 값: 0.0060
에포크 22/100 진행 중...
에포크 22 완료, 평균 손실 값: 0.0047
에포크 23/100 진행 중...
에포크 2

#### 8. 모델 저장
- 학습이 완료된 모델과 토크나이저를 저장

In [9]:
model.save_pretrained("./food_chatbot_model")
tokenizer.save_pretrained("./food_chatbot_model")


('./food_chatbot_model\\tokenizer_config.json',
 './food_chatbot_model\\special_tokens_map.json',
 './food_chatbot_model\\vocab.json',
 './food_chatbot_model\\merges.txt',
 './food_chatbot_model\\added_tokens.json',
 './food_chatbot_model\\tokenizer.json')

#### 9. 챗봇 실행을 위한 모델 로드
- 저장된 모델과 토크나이저를 로드하여 챗봇을 실행할 준비

In [10]:
from transformers import GPT2LMHeadModel, AutoTokenizer

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("./food_chatbot_model")
model = GPT2LMHeadModel.from_pretrained("./food_chatbot_model")
model.to(device)
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

#### 10. 챗봇 응답 생성 함수 정의
- 챗봇의 응답을 생성하는 함수를 정의
- 여기서 Zero-Shot과 Few-Shot 학습 방식을 적용
##### 설명:
- Zero-Shot: 예시 없이 직접 질문에 답변할 때 사용, 모델의 사전 학습된 지식을 활용
- Few-Shot: 몇 가지 예시를 제공하여 모델이 응답의 패턴을 학습하도록 유도 
    - 이를 통해 응답의 일관성과 정확성을 높일 수 있다.

- max_length 설정: tokenizer.encode()와 model.generate()에서 max_length를 명시적으로 설정하여 경고를 방지하고, 입력 시퀀스의 최대 길이 제한
- 응답 추출: 답변: 이후의 텍스트를 추출하여 반환,
    - 만약 답변:이 없을 경우, 기본적인 응답을 제공
- 디버깅 추가: output을 출력하여 모델이 어떤 텍스트를 생성하고 있는지 확인

In [11]:
def generate_response(input_text, mode='few-shot'):
    if mode == 'zero-shot':
        prompt = f"질문: {input_text}\n답변: "
    elif mode == 'few-shot':
        # Few-Shot을 위해 몇 가지 예시를 추가
        few_shot_examples = (
            "질문: 배가 고픈데 뭐 먹을까?\n답변: 한식은 어때요? 따뜻한 국밥이 좋을 것 같아요.\n"
            "질문: 점심 추천해줘\n답변: 가벼운 샐러드나 샌드위치는 어떠세요?\n"
            "질문: 저녁 뭐 먹을지 고민돼\n답변: 맛있는 파스타나 스테이크는 어떠신가요?\n"
        )
        prompt = f"{few_shot_examples}질문: {input_text}\n답변: "
    else:
        prompt = f"질문: {input_text}\n답변: "

    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=128).to(device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=128,  # 적절한 max_length 설정
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.92,
            temperature=0.6,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )
    output = tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True  # 경고를 제거하기 위해 추가
    )
    
    # 디버깅을 위한 출력
    print(f"DEBUG: Generated Output: {output}")

    # 답변 부분만 추출 (마지막 '답변:' 이후 텍스트)
    if "답변:" in output:
        response = output.rsplit("답변:", 1)[-1].strip()
    else:
        response = "죄송합니다. 이해하지 못했어요."

    return response


#### 11. 챗봇 실행
- 챗봇을 실행하여 질문에 답변 처리
##### 설명:
- 종료 조건: 사용자가 '종료'를 입력하면 대화를 종료
- 응답 생성: generate_response 함수에 mode를 전달하여 Zero-Shot 또는 Few-Shot 방식을 선택
    - few-shot 방식을 사용하도록 설정

In [13]:
print("음식 추천 챗봇입니다. 종료하려면 '종료'를 입력하세요.")

while True:
    user_input = input("사용자: ")
    if user_input.strip().lower() == "종료":
        print("챗봇: 대화를 종료합니다. 좋은 하루 되세요!")
        break
    # mode를 'zero-shot' 또는 'few-shot'으로 설정
    response = generate_response(user_input, mode='zero-shot')
    print(f"챗봇: {response}")


음식 추천 챗봇입니다. 종료하려면 '종료'를 입력하세요.
DEBUG: Generated Output: 질문: 점심추천해줘
답변: 짬밥이나 불고기를 추천합니다.
챗봇: 짬밥이나 불고기를 추천합니다.
DEBUG: Generated Output: 질문: 짬밥말고 다른거
답변: 짬뽕이나 불닭볶음면은 어떠세요?
챗봇: 짬뽕이나 불닭볶음면은 어떠세요?
DEBUG: Generated Output: 질문: 다른거
답변: 칵테나 레몬 에이드를 드셔보세요.
챗봇: 칵테나 레몬 에이드를 드셔보세요.
DEBUG: Generated Output: 질문: 점심에 레모?
답변: 짬밥이나 샌드위치를 드셔보세요.
챗봇: 짬밥이나 샌드위치를 드셔보세요.
DEBUG: Generated Output: 질문: 다른거 없나?
답변: 쫄깃한 국밥이 좋을 것 같아요.
챗봇: 쫄깃한 국밥이 좋을 것 같아요.
챗봇: 대화를 종료합니다. 좋은 하루 되세요!
